In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
input_csv=input('Enter the csv file name (for chain names / saving files): ')

Enter the csv file name (for chain names / saving files):  mahe_data


In [3]:
bus_df=pd.read_csv(f'{input_csv}.csv')
bus_df

,College/Institute,Vendor Name,Vehicle Number,Route No,Driver Point Latitude,Driver Point Longitude,Driver Point Name,First Pickup Pt. Latitude,First Pickup Pt. Longitude,First Pickup Point Name,Seat Capacity,dead km,osrm distance,dev
0,MAHE,Baghirathi,KA53D5162,M1,12.855323,77.672600,Veerasagara Village,12.907820,77.604260,IIM-B,52,12.9,13.0193,-0.1193
1,MAHE,Baghirathi,KA53D0166,M2,13.069991,77.603710,Kodigehalli,12.955736,77.592776,Shantinagar Bus Terminal,54,17.5,15.9934,1.5066
2,MAHE,Baghirathi,KA53B6965,M3,13.123265,77.592183,MAHE,13.049924,77.592684,Esteem Mall Bus Stop,50,11.2,13.3715,-2.1715
3,MAHE,Baghirathi,KA53D8420,M4,12.924835,77.649409,HSR Layout,12.917237,77.623038,Silk Board Junction,52,3.9,4.3638,-0.4638
4,MAHE,Baghirathi,KA53D4606,M5,12.894084,77.533930,Subramanyapura,12.937688,77.519786,RR Nagar Arch,52,7.6,7.6461,-0.0461
5,MAHE,Baghirathi,KA53D8419,M6,13.137676,77.491792,IIHR Hesaraghatta,13.048166,77.500035,Nagasandra Metro Station,52,14.0,13.8252,0.1748
6,MAHE,Baghirathi,KA53D5165,M7,12.997318,77.503035,Hegganahalli,12.998231,77.553015,Navarang Circle,52,9.0,6.9961,2.0039
7,MAHE,Baghirathi,KA53D0158,M8,13.083934,77.560430,Doddabettahalli,13.046461,77.555852,BEL Circle,54,7.0,5.9380,1.0620
8,MAHE,Baghirathi,KA07A9086,M9,12.996444,77.716609,Hoodi,12.969909,77.750179,Whitefield Bus Stop,51,6.9,6.4263,0.4737
9,MAHE,Baghirathi,KA53D2958,M10,12.996542,77.716401,Hoodi,13.019929,77.709581,Bhattarahalli Bus Stop,52,6.0,4.0491,1.9509


In [4]:
driver_df=bus_df[['Vehicle Number', 'Route No', 'Driver Point Latitude', 'Driver Point Longitude', 'Driver Point Name', 'College/Institute']].rename(columns={
    'Vehicle Number':'bus', 
    'Route No': 'route',
    'Driver Point Latitude': 'dlat',
    'Driver Point Longitude': 'dlon',
    'Driver Point Name': 'dname',
    'College/Institute': 'cname'
})
pickup_df=bus_df[['Vehicle Number', 'Route No', 'First Pickup Pt. Latitude', 'First Pickup Pt. Longitude', 'First Pickup Point Name', 'College/Institute']].rename(columns={
    'Vehicle Number':'bus', 
    'Route No': 'route',
    'First Pickup Pt. Latitude': 'plat',
    'First Pickup Pt. Longitude': 'plon',
    'First Pickup Point Name': 'pname',
    'College/Institute': 'cname'
})
driver_df=driver_df.set_index('bus')
pickup_df=pickup_df.set_index('bus')

In [5]:
driver_df.head()
pickup_df.head()

,route,plat,plon,pname,cname
bus,,,,,
KA53D5162,M1,12.907820,77.604260,IIM-B,MAHE
KA53D0166,M2,12.955736,77.592776,Shantinagar Bus Terminal,MAHE
KA53B6965,M3,13.049924,77.592684,Esteem Mall Bus Stop,MAHE
KA53D8420,M4,12.917237,77.623038,Silk Board Junction,MAHE
KA53D4606,M5,12.937688,77.519786,RR Nagar Arch,MAHE


In [6]:
api_token='pk.eyJ1'
base_URL= "https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"

In [8]:
driver_coords = list(zip(driver_df['dlat'], driver_df['dlon']))
pickup_coords = list(zip(pickup_df['plat'], pickup_df['plon']))
driver_coords 
pickup_coords

[(12.90782, 77.60426),
 (12.95573552, 77.59277575),
 (13.049924, 77.592684),
 (12.917237, 77.623038),
 (12.93768849, 77.51978608),
 (13.04816628, 77.50003502),
 (12.99823052, 77.5530152),
 (13.046461, 77.555852),
 (12.96990935, 77.75017944),
 (13.01992925, 77.70958107),
 (13.043178, 77.613378),
 (12.9252461, 77.64906656),
 (13.21327379, 77.55474242),
 (13.10627501, 77.57654828),
 (13.0841, 77.595852)]

In [ ]:
N = len(driver_coords)
assert N == len(pickup_coords), "Drivers and pickups must match!"

full_matrix = np.full((N, N), np.nan)  # Final NxN matrix

# Chunking function
def chunks(lst, size):
    for i in range(0, len(lst), size):
        yield i, lst[i:i + size]

# Build combined coordinates string from (lat, lon) → lon,lat
def build_coords_string(points):
    return ";".join([f"{lon},{lat}" for lat, lon in points])

max_total_coords = 25
driver_chunk_size = 24

for di, d_chunk in chunks(driver_coords, driver_chunk_size):
    for pi, p_chunk in chunks(pickup_coords, 1):  # process one pickup at a time
        combined_points = d_chunk + p_chunk

        # Make sure we don't exceed 25 total points
        if len(combined_points) > max_total_coords:
            raise ValueError(f"Too many coordinates: {len(combined_points)} combined points exceed Mapbox limit.")

        sources = ";".join(str(i) for i in range(len(d_chunk)))
        destinations = ";".join(str(len(d_chunk) + j) for j in range(len(p_chunk)))

        coords_str = build_coords_string(combined_points)

        url = (
            f"{base_URL}{coords_str}"
            f"?sources={sources}&destinations={destinations}"
            f"&annotations=distance"
            f"&access_token={api_token}"
        )

        try:
            resp = requests.get(url)
            json_data = resp.json()

            if "distances" not in json_data:
                print(f"\n🚨 Missing 'distances' for drivers {di}-{di+len(d_chunk)} pickup {pi}")
                print("Raw response:", json_data)
                continue

            distances = np.array(json_data["distances"]) / 1000.0  # meters → km

            # Insert distances into the correct rows and single column of the final matrix
            full_matrix[
                di:di+len(d_chunk),
                pi:pi+len(p_chunk)
            ] = distances

            print(f"✅ Filled drivers {di}-{di+len(d_chunk)-1}, pickup {pi}")
        
        except Exception as e:
            print(f"\n❌ Exception for drivers {di}-{di+len(d_chunk)} pickup {pi}: {e}")
df = pd.DataFrame(full_matrix)
df